## Setup

In [ ]:
from os import cpu_count

In [ ]:
import logging

from report_generators.reporters import CountryReport, GermanyReport, USAReport, HungaryReport, AllRegions
from report_generators.executors import ReportExecutor
from report_generators.index import create_markdown_index_page

In [ ]:
#  Set to false if you do not want multiprocessing enabled
workers = 'auto'

if workers == 'auto':
    workers = max(1, cpu_count())
    # try at most 4 to reduce probability of error message like
    # the one shown at https://github.com/jupyter/jupyter_client/issues/541
    workers = max(workers-2, 1)


if workers:
    print(f'Using {workers} processes')
    
wwwroot = "wwwroot"

In [ ]:
logging.basicConfig(
    format="%(asctime)s %(threadName)s: %(message)s",
    level=logging.WARNING,
    datefmt="%H:%M:%S"
)

In [ ]:
%config InlineBackend.figure_formats = ['svg']

from oscovida import *
from oscovida import MetadataRegion

pd.set_option('display.float_format', '{:.2f}'.format)  #  Disable pandas scientific notation

Cleaning of cache and copying files has moved to

- `generate-webpage-clean-setup.py` and 
- `generate-webpage-clean-setup.sh`

### Download Datasets

In [ ]:
d, c = fetch_deaths(), fetch_cases()

countries = d.index
countries2 = c.index
assert (countries2 == countries).all()

In [ ]:
data_US_cases = fetch_cases_US()
data_US_deaths = fetch_deaths_US()

In [ ]:
# also fetch data from Germany, so it is available later from the cache
germany = fetch_data_germany()

In [ ]:
hungary = fetch_data_hungary()

### Generic Functions

In [ ]:
def does_wwwroot_exist(wwwroot, create=False):
    if not os.path.exists(wwwroot):
        msg = "To put the html into github repo for webhosting, run "
        msg += '"git clone git@github.com:oscovida/oscovida.github.io.git wwwroot" or similar'
        if create:
            os.mkdir(wwwroot)
            os.mkdir(wwwroot+"/ipynb")
            os.mkdir(wwwroot+"/html")
        else:
            raise ValueError(f"directory {wwwroot} missing.")

In [ ]:
does_wwwroot_exist(wwwroot, create=True)

## Country Report Generation

In [ ]:
def get_country_list():
    d, c = fetch_deaths(), fetch_cases()

    countries = d.index
    countries2 = c.index
    assert (countries2 == countries).all()
    
    # Here we should identify regions in countries, and process those.
    # Instead, as a quick hack to get started, we'll just take one country
    # and the current "get_country" method will sum over all regions of one country if only 
    # the country name is given.
    
    return sorted(countries.drop_duplicates())

In [ ]:
countries = get_country_list()

In [ ]:
cre = ReportExecutor(Reporter=CountryReport,
    wwwroot=wwwroot, expiry_hours=2, attempts=3, workers=workers, force=True)

In [ ]:
cre.create_html_reports(countries)

In [ ]:
cre.create_markdown_index_page()

## Germany Report Generation

In [ ]:
def get_germany_regions_list():
    data_germany = fetch_data_germany()
    land_kreis = data_germany[['Bundesland', 'Landkreis']]
    ordered = land_kreis.sort_values(['Bundesland', 'Landkreis'])
    return ordered.drop_duplicates().values.tolist()

In [ ]:
germany_regions = get_germany_regions_list()
wwwroot = "wwwroot"

# data cleaning: on 13 April, we had a Landkreis "LK Göttingen (alt)"
# with only one data point. This causes plots to fail, because there
# is nothing to plot, and then the legend() command failed.
# We assume that the RKI labels unusual data with '(alt)', and remove those.

alt_data_sets = ["(alt)" in r[1].lower() for r in germany_regions]
if sum(alt_data_sets) > 0:
    bad_datasests = list(compress(germany_regions, alt_data_sets))
    
    print(f"Removing datasets label with '(alt)': {bad_datasests}")

    for bd in bad_datasests:
        c, d, _ = germany_get_region(landkreis=bd[1])
        print(f"\tremoved: {bd} : len(cases)={len(c)}, len(deaths)={len(d)}")

    bad_indecies = list(compress(range(len(alt_data_sets)), alt_data_sets))

    [germany_regions.pop(i) for i in bad_indecies]

In [ ]:
gre = ReportExecutor(Reporter=GermanyReport,
    wwwroot=wwwroot, expiry_hours=2, attempts=3, workers=workers, force=True)

In [ ]:
gre.create_html_reports(germany_regions)

In [ ]:
gre.create_markdown_index_page()

## USA Report Generation

In [ ]:
states = get_US_region_list()

In [ ]:
usre = ReportExecutor(Reporter=USAReport,
    wwwroot=wwwroot, expiry_hours=2, attempts=3, workers=workers, force=True)

In [ ]:
usre.create_html_reports(states)

In [ ]:
usre.create_markdown_index_page()

## Hungary Report Generation

In [ ]:
counties = get_counties_hungary()

In [ ]:
hre = ReportExecutor(Reporter=HungaryReport,
    wwwroot=wwwroot, expiry_hours=2, attempts=3, workers=workers, force=True)

In [ ]:
hre.create_html_reports(counties)

In [ ]:
hre.create_markdown_index_page()

### HTML Pages for All Regions

In [ ]:
arre = ReportExecutor(Reporter=AllRegions, wwwroot=wwwroot)

In [ ]:
arre.create_markdown_index_page()

## Error Reporting

In [ ]:
ms = MetadataRegion.get_all()
for name in ms:
    m = MetadataRegion(name)
    dt = m.last_updated_hours_ago()
    if dt > 2:
        print(f"Problem with '{name}', last update: {dt} ago ")